In [ ]:
import igraph as ig
import pandas as pd
import numpy as np
from pathlib import Path
import scipy.sparse as sp
from src.config import INTERIM_DATA_DIR

## 读取图结构

In [ ]:
# 图结构位置
merged_graph_path = INTERIM_DATA_DIR / "Texas7k_Gas/merged_graph.pkl"

# 读取图结构
g = ig.Graph.Read_Pickle(merged_graph_path)

# 获取所有节点和边的信息
nodes = g.vs  # 节点列表
edges = g.es  # 边列表

# 统计图结构信息
print(f"总节点数: {g.vcount()}")
print(f"总边数: {g.ecount()}")

## 灾害模拟

In [ ]:
"""
灾害发生模型
使用非齐次泊松过程模拟灾害发生的时间，假设每个灾害是独立的，且符合泊松分布。
"""
def poisson_process(lambda_rate, T):
    """
    使用泊松过程生成灾害发生的时间
    lambda_rate: 每年发生灾害的平均次数
    T: 模拟时间范围
    """
    event_times = []
    t = 0
    while t < T:
        t += np.random.exponential(1 / lambda_rate)
        if t < T:
            event_times.append(t)
    return np.array(event_times)

In [ ]:
# 假设地震的平均发生率为0.01次/年，模拟100年的时间范围
lambda_rate = 0.01
T = 100  # 模拟100年
earthquake_times = poisson_process(lambda_rate, T)

print("地震事件发生的时间:", earthquake_times)

In [ ]:
for time in earthquake_times:
    g.add_vertex(f"Earthquake_{time}")
    g.vs[-1]["time"] = time  # 地震事件发生的时间
    g.vs[-1]["severity"] = np.random.normal(0.3, 0.1)  # 地震的严重性 (PGA)

In [ ]:
"""
灾害严重性模型
假设灾害(地震)的严重性服从正态分布,即峰值地面加速度(PGA)服从正态分布。
"""
def generate_severity(num_events, mean=0.3, std_dev=0.1):
    """
    num_events: 需要模拟的事件数量
    mean: 正态分布的均值(PGA均值)
    std_dev: 正态分布的标准差
    """
    severity = np.random.normal(loc=mean, scale=std_dev, size=num_events)
    return severity


In [ ]:
# 假设我们有10次地震事件
num_events = len(earthquake_times)
earthquake_severity = generate_severity(num_events)

print("地震事件的严重性:", earthquake_severity)

In [ ]:
"""
定义灾害之间的触发关系（例如，地震引发海啸或电力中断），
假设地震的严重性高于某个阈值时，可能会影响电力和天然气网络的节点。
"""
def trigger_secondary_hazards(earthquake_severity, threshold=0.5):
    """
    假设地震发生时，若地震强度超过某个阈值（例如0.5g），则会触发次生灾害（例如电力中断、天然气泄漏）
    """
    triggered_hazards = []
    for severity in earthquake_severity:
        if severity >= threshold:
            triggered_hazards.append(True)  # 触发了次生灾害
        else:
            triggered_hazards.append(False)  # 没有触发次生灾害
    return triggered_hazards


In [ ]:
# 判断哪些地震事件会触发次生灾害
triggered_hazards = trigger_secondary_hazards(earthquake_severity, threshold=0.5)
print("哪些地震事件触发了次生灾害:", triggered_hazards)

In [ ]:
def create_disaster_graph(earthquake_times, earthquake_severity, triggered_hazards):
    """
    创建一个多灾害图，表示地震、次生灾害等灾害之间的相互关系
    earthquake_times: 地震事件发生的时间
    earthquake_severity: 地震事件的严重性（PGA）
    triggered_hazards: 哪些灾害触发了次生灾害
    """
    # 创建一个空的图
    g_disaster = ig.Graph(directed=True)

    # 添加灾害事件节点（例如地震）
    g_disaster.add_vertices(len(earthquake_times) + 1)  # 包含一个海啸节点（假设）
    
    for i in range(len(earthquake_times)):
        g_disaster.vs[i]['time'] = earthquake_times[i]
        g_disaster.vs[i]['severity'] = earthquake_severity[i]
    
    # 添加海啸节点
    g_disaster.vs[-1]['time'] = "Triggered by Earthquake"
    g_disaster.vs[-1]['severity'] = "High"  # 海啸的强度假设为高
    
    # 根据地震的触发关系添加边
    for i, triggered in enumerate(triggered_hazards):
        if triggered:
            g_disaster.add_edge(i, len(g_disaster.vs) - 1)  # 从地震到海啸的边
    
    return g_disaster

# 创建多灾害图
g_disaster = create_disaster_graph(earthquake_times, earthquake_severity, triggered_hazards)

# 输出图的节点和边
print("图的节点:", g_disaster.vs['time'])
print("图的边:", g_disaster.es())

In [ ]:
def simulate_disaster_impact(g, earthquake_times, earthquake_severity, triggered_hazards, threshold=0.5):
    """
    模拟灾害对电力和天然气网络的影响，假设灾害会影响某些节点和边的状态
    earthquake_times: 地震事件的发生时间
    earthquake_severity: 地震事件的严重性
    triggered_hazards: 哪些灾害触发了次生灾害
    """
    # 遍历所有地震事件，评估其对网络节点的影响
    for i, triggered in enumerate(triggered_hazards):
        if triggered:
            # 获取受影响的电力节点和天然气节点
            affected_nodes = g.vs.select(_degree_ge=1)  # 假设所有度大于1的节点都受影响
            for node in affected_nodes:
                # 标记受影响节点（例如，损坏）
                node['status'] = 'damaged'
    
    return g

# 模拟灾害对网络的影响
g_affected = simulate_disaster_impact(g, earthquake_times, earthquake_severity, triggered_hazards)

# 输出受影响的节点
affected_nodes = [node.index for node in g_affected.vs if node['status'] == 'damaged']
print("受影响的节点索引:", affected_nodes)